<a href="https://colab.research.google.com/github/swat08mx/CONVEX/blob/main/AMR_NER_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import time
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
import seaborn as sn
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from gensim.models import KeyedVectors, word2vec
import gensim as gs
from sklearn.manifold import TSNE
import json
from spacy.training import offsets_to_biluo_tags
from spacy.lang.en import English
# from nltk import pos_tag, word_tokenize, sent_tokenize, wordpunct_tokenize
# nltk.download()

In [ ]:
data = pd.read_json("project_9_dataset.jsonl", lines=True)
data

,id,text,annotations,meta,annotation_approver,comments
0,685,Current limitations in the understanding and c...,"[{'label': 35, 'start_offset': 159, 'end_offse...",{},shantanu,[]
1,686,"Without addressing gaps in identified areas, s...",[],{},shantanu,[]
2,1893,Thirty-eight clones were detected.,[],{},shantanu,[]
3,670,Several examples of the implementation of DART...,"[{'label': 37, 'start_offset': 42, 'end_offset...",{},shantanu,[]
4,688,"Specifically, antimicrobial use in human, anim...",[],{},shantanu,[]
...,...,...,...,...,...,...
2332,2329,"In this review, we summarize the status of res...","[{'label': 33, 'start_offset': 240, 'end_offse...",{},shantanu,[]
2333,2473,"Here, we survey the current literature focussi...",[],{},shantanu,[]
2334,2276,The results showed that the number of related ...,[],{},shantanu,[]
2335,1071,We also describe the current state of developm...,[],{},shantanu,[]


In [ ]:
## code for making lists of datas and storing info
text=[]
start=[]
end=[]
label=[]
final=[]

for i in range(len(data)-1):
  if data['annotations'][i]!=[]:
    if len(data['annotations'][i])>0:
      for j in range(len(data['annotations'][i])):
        text.append(data['text'][i])
        start.append(data['annotations'][i][j]['start_offset'])
        end.append(data['annotations'][i][j]['end_offset'])
        label.append(data['annotations'][i][j]['label'])
  else:
    text.append(data['text'][i])
    start.append("")
    end.append("")
    label.append("")

Label Parser

In [ ]:
labels_data = pd.read_json("project_9_labels.json")
dat=36
for i in range(len(labels_data)-1):
  if labels_data['id'][i]==dat:
    dat=labels_data['text'][i]

#label swapper code
for i in range(len(label)):
  for j in range(len(labels_data)):
    if label[i]==labels_data['id'][j]:
      label[i]=labels_data['text'][j]

In [ ]:
##### bug fix code for space errors
for i in range(len(text)):
  try:
    start_new = int(start[i])
    end_new = int(end[i])
    print(f"The {i} line is : {text[i][start_new:end_new]}")
    var=text[i][start_new:end_new]
    bef=len(var)
    print(bef)
    var_one=var.strip()
    print(var_one)
    aft=len(var_one)
    if bef!=aft:
      if var[-1]==" ":
        end[i]-=1
      else:
        start[i]+=1
    end_new=end[i]
    start_new=start[i]
    print(len(text[i][start_new:end_new]))
    print("-----------------------------------------------")
  except:
    pass

Streaming output truncated to the last 5000 lines.
The 1583 line is : Spn
3
Spn
3
-----------------------------------------------
The 1584 line is : Mega1. IV or Mega2. IVc
23
Mega1. IV or Mega2. IVc
23
-----------------------------------------------
The 1585 line is : Vibrio
6
Vibrio
6
-----------------------------------------------
The 1586 line is : biochemical tests
17
biochemical tests
17
-----------------------------------------------
The 1587 line is : molecular detection
19
molecular detection
19
-----------------------------------------------
The 1588 line is : pyrH
4
pyrH
4
-----------------------------------------------
The 1590 line is : norfloxacin
11
norfloxacin
11
-----------------------------------------------
The 1591 line is : penicillin G
12
penicillin G
12
-----------------------------------------------
The 1604 line is : MRSA
4
MRSA
4
-----------------------------------------------
The 1605 line is : (XY)2RRRF(YX)2-NH2
18
(XY)2RRRF(YX)2-NH2
18
---------------------

In [ ]:
for i in range(len(text)):
  print("{}: [({}, {}, {}, {})]".format(i,start[i],end[i],label[i],text[i]))

0: [(159, 182, Techniques, Current limitations in the understanding and control of antimicrobial resistance (AMR) in Canada are described through a comprehensive review focusing on: (1) treatment optimization; (2) surveillance of antimicrobial use and AMR; and (3) prevention of transmission of AMR.)]
1: [(, , , Without addressing gaps in identified areas, sustained progress in AMR mitigation is unlikely. Expert opinions and perspectives contributed to prioritizing identified gaps.)]
2: [(, , , Thirty-eight clones were detected.)]
3: [(42, 51, Others, Several examples of the implementation of DART 2020 are outlined here.)]
4: [(, , , Specifically, antimicrobial use in human, animal, crop, and environmental sectors cannot be regarded as independent; therefore, a One Health approach is needed in AMR research and understanding, current surveillance efforts, and policy.)]
5: [(, , , Antimicrobial resistance (AMR) is tracked most closely in clinical settings and high-income countries.)]
6: [

In [ ]:
len(start), len(end), len(label), len(text)

(3427, 3427, 3427, 3427)

In [ ]:
text[33][183:197]
end[33]

82

In [ ]:
#code for printing tagged sentences
## -------------- problem is that un-annotated sentences are returning errors instead of tagging O to all words ------------
res=[]
for i in range(1000):
  start_temp=start[i:i+1]
  end_temp=end[i:i+1]
  label_temp=label[i:i+1]
  result=list(zip(start_temp, end_temp, label_temp))
  res.append(result)
  nlp=English()
  offsets = res[i]
  doc = nlp(text[i])
  try:
    tags = offsets_to_biluo_tags(nlp.make_doc(text[i]), offsets)
    print([token.text for token in doc])
    print(tags)
  except:
    pass


['Current', 'limitations', 'in', 'the', 'understanding', 'and', 'control', 'of', 'antimicrobial', 'resistance', '(', 'AMR', ')', 'in', 'Canada', 'are', 'described', 'through', 'a', 'comprehensive', 'review', 'focusing', 'on', ':', '(', '1', ')', 'treatment', 'optimization', ';', '(', '2', ')', 'surveillance', 'of', 'antimicrobial', 'use', 'and', 'AMR', ';', 'and', '(', '3', ')', 'prevention', 'of', 'transmission', 'of', 'AMR', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Techniques', 'I-Techniques', 'L-Techniques', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Several', 'examples', 'of', 'the', 'implementation', 'of', 'DART', '2020', 'are', 'outlined', 'here', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'B-Others', 'L-Others', 'O', 'O', 'O', 'O']
['We', 'report', 'the', 'use', 'of', 'a', 'novel', 'pain', '-', 'relief', 'therapy', 'for', 'FMD

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Phenotypic resistance of E. coli against penicilli..." with entities "[(237, 248, 'Antibiotics')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


['Phenotypic', 'resistance', 'of', 'E.', 'coli', 'against', 'penicillins', 'and', 'beta', '-', 'lactamase', 'inhibitors', '(', '20.2%-100', '%', ')', ',', 'cephalosporins', '(', '1.9%-100', '%', ')', ',', 'fluoroquinolones', '(', '5.98%-100', '%', ')', ',', 'aminoglycosides', '(', '6%-100', '%', ')', ',', 'tetracyclines', '(', '17.7%-100', '%', ')', ',', 'carbapenems', '(', '13.6%-72.7', '%', ')', ',', 'macrolides', '(', '11.8%-100', '%', ')', ',', 'polymyxins', '(', '7.9%-100', '%', ')', ',', 'phenicols', '(', '20%-97.2', '%', ')', ',', 'sulfa', 'drugs', '(', '44.7%-100', '%', ')', ',', 'cephamycins', '(', '21.4%-48.8', '%', ')', ',', 'nitrofurans', '(', '21.4%-63.2', '%', ')', ',', 'monobactams', '(', '1.2', '%', ')', ',', 'and', 'glycylcyclines', '(', '2.3', '%', ')', 'was', 'recorded', 'in', 'the', 'last', 'decades', 'in', 'Bangladesh', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Phenotypic resistance of E. coli against penicilli..." with entities "[(286, 296, 'Antibiotics')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Phenotypic resistance of E. coli against penicilli..." with entities "[(309, 321, 'Antibiotics')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


['Phenotypic', 'resistance', 'of', 'E.', 'coli', 'against', 'penicillins', 'and', 'beta', '-', 'lactamase', 'inhibitors', '(', '20.2%-100', '%', ')', ',', 'cephalosporins', '(', '1.9%-100', '%', ')', ',', 'fluoroquinolones', '(', '5.98%-100', '%', ')', ',', 'aminoglycosides', '(', '6%-100', '%', ')', ',', 'tetracyclines', '(', '17.7%-100', '%', ')', ',', 'carbapenems', '(', '13.6%-72.7', '%', ')', ',', 'macrolides', '(', '11.8%-100', '%', ')', ',', 'polymyxins', '(', '7.9%-100', '%', ')', ',', 'phenicols', '(', '20%-97.2', '%', ')', ',', 'sulfa', 'drugs', '(', '44.7%-100', '%', ')', ',', 'cephamycins', '(', '21.4%-48.8', '%', ')', ',', 'nitrofurans', '(', '21.4%-63.2', '%', ')', ',', 'monobactams', '(', '1.2', '%', ')', ',', 'and', 'glycylcyclines', '(', '2.3', '%', ')', 'was', 'recorded', 'in', 'the', 'last', 'decades', 'in', 'Bangladesh', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

...

['For', 'risk', 'assessment', 'purposes', 'related', 'to', 'human', 'health', ',', 'AMR', 'plasmids', 'found', 'in', 'food', 'and', 'bearing', 'genes', 'conferring', 'resistances', 'to', 'critically', 'important', 'antibiotics', 'in', 'human', 'medicine', '-', 'such', 'as', 'to', 'extended', '-', 'spectrum', 'cephalosporins', ',', 'carbapenems', 'or', 'colistin', '-', 'have', 'been', 'under', 'specific', 'scrutiny', 'these', 'last', 'years', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-Antibiotics', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['For', 'risk', 'assessment', 'purposes', 'related', 'to', 'human', 'health', ',', 'AMR', 'plasmids', 'found', 'in', 'food', 'and', 'bearing', 'genes', 'conferring', 'resistances', 'to', 'critically', 'important', 'antibiotics', 'in', 'human', 'medicine', '-', 'such', 'as', 'to', 'extended', '-'

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Currently, doxycycline, azithromycin, or ciproflox..." with entities "[(40, 54, 'Antibiotics')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Currently, doxycycline, azithromycin, or ciproflox..." with entities "[(203, 217, 'Antibiotics')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


['Currently', ',', 'doxycycline', ',', 'azithromycin', ',', 'or', 'ciprofloxacin', 'are', 'commonly', 'used', 'for', 'V.', 'cholerae', ',', 'and', 'doxycycline', 'or', 'quinolone', 'are', 'administered', 'for', 'V.', 'parahaemolyticus', ',', 'whereas', 'doxycycline', 'and', 'a', 'third', '-', 'generation', 'cephalosporin', 'are', 'recommended', 'for', 'V.', 'vulnificus', 'as', 'initial', 'treatment', 'regimen', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', '-', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Currently', ',', 'doxycycline', ',', 'azithromycin', ',', 'or', 'ciprofloxacin', 'are', 'commonly', 'used', 'for', 'V.', 'cholerae', ',', 'and', 'doxycycline', 'or', 'quinolone', 'are', 'administered', 'for', 'V.', 'parahaemolyticus', ',', 'whereas', 'doxycycline', 'and', 'a', 'third', '-', 'generation', 'cephalosporin', 'are', 'recommended', 'for', 'V.', 'vul

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Of these, those caused by diarrheagenic Escherichi..." with entities "[(39, 56, 'Antibiotic-Resistant Bacteria')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


['In', '2009', ',', 'cephem', '-', 'resistant', 'isolates', 'were', 'detected', 'in', 'children', 'as', 'well', 'as', 'a', 'teacher', '.']
['O', 'O', 'O', 'U-Antibiotics', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Pulsed', '-', 'field', 'gel', 'electrophoresis', 'revealed', '100', '%', 'similarity', 'for', 'four', 'isolates', 'in', '2007', 'and', 'one', 'isolate', 'in', '2008', ',', 'and', 'also', 'similarity', 'among', 'seven', 'isolates', 'carrying', 'the', 'virulence', 'gene', '(', 'CNF1', ')', '.']
['O', 'O', 'O', 'B-Techniques', 'L-Techniques', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Laboratory', '-', 'based', 'AMR', 'testing', 'and', 'characterization', 'is', 'the', 'key', 'component', 'of', 'an', 'AMR', 'surveillance', 'program', '.']
['O', 'O', 'O', 'B-Techniques', 'L-Techniques', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Throug

In [ ]:
lm="Whilst K. pneumoniae's contribution to the overall AMR crisis is impossible to quantify, current evidence suggests it has a wider ecological distribution, significantly more varied DNA composition, greater AMR gene diversity and a higher plasmid burden than other Gram negative opportunists."
te= lm[7:21]
te

"K. pneumoniae'"

In [ ]:
res[950], text[950][12:32]

([(13, 32, 'Techniques')], ' gel electrophoresis')

In [ ]:
from spacy.training import offsets_to_biluo_tags
from spacy.lang.en import English
nlp=English()
offsets = res[950]
doc = nlp(text[950])
tags = offsets_to_biluo_tags(doc, offsets)
print([token.text for token in doc])
print(tags)

['Pulsed', '-', 'field', 'gel', 'electrophoresis', 'revealed', '100', '%', 'similarity', 'for', 'four', 'isolates', 'in', '2007', 'and', 'one', 'isolate', 'in', '2008', ',', 'and', 'also', 'similarity', 'among', 'seven', 'isolates', 'carrying', 'the', 'virulence', 'gene', '(', 'CNF1', ')', '.']
['O', 'O', 'O', 'B-Techniques', 'L-Techniques', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
labels_data

,id,text,prefix_key,suffix_key,background_color,text_color
0,31,Gene/Proteins,NaN,0,#1A237E,#ffffff
1,32,Microbes,NaN,1,#76F648,#ffffff
2,33,Antibiotic-Resistant Bacteria,NaN,2,#07360A,#ffffff
3,34,Antibiotic-Resistant Genes,NaN,3,#EFF1EF,#ffffff
4,35,Techniques,NaN,4,#D51D0F,#ffffff
5,36,Antibiotics,NaN,5,#159EE1,#ffffff
6,37,Others,NaN,6,#D4C543,#ffffff


In [ ]:
# dicto=[]
# str_one=" "
# for i in range(2337):
#   dicto.append(data_two['text'][i])
# listToStr = ' '.join([str(elem) for elem in dicto])
# sentences=nltk.sent_tokenize(listToStr)
# data_three = [(i+1, nltk.word_tokenize(sentence)) for i, sentence in enumerate(sentences)]
# words = [(sentence, word) for sentence, words in data_three for word in words]
# df = pd.DataFrame(words, columns=['Sentence', 'Word'])

In [ ]:
df

In [ ]:
word_counts = df.groupby(["Sentence"])['Word'].agg(['count'])
word_counts.hist(bins=50)

In [ ]:
all_words = list(df['Word'].unique())
word2index = {word : idx for idx, word in enumerate(all_words, 2)}
word2index['--UNK--']=0
word2index['--PAD--']=1
word2index

In [ ]:
lk = " gene therap"
lk=lk.strip()
lk[-1]

'p'